In [2]:
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera

car = NvidiaRacecar()
camera = CSICamera(width=224, height=224, capture_fps=15)
camera.running=True

WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


RuntimeError: Could not initialize camera.  Please see error trace.

In [1]:
import cv2
import torch
from torchvision.models import resnet18
from torch2trt import TRTModule
import torch.nn.functional as F
import torchvision.transforms as transforms
import PIL.Image
import numpy as np
import time

In [3]:
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_model_resnet18_trt.pth'))

<All keys matched successfully>

In [4]:
device = torch.device('cuda')
model = model_trt.to(device)
model = model_trt.eval().half()

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [8]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetcam.utils import bgr8_to_jpeg

image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
speed_slider = widgets.FloatSlider(description='speed', min=0.0, max=0.5, value=0.0, step=0.01, orientation='horizontal')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.VBox([widgets.HBox([image, blocked_slider]), speed_slider]))

In [6]:
def update(change):
    global blocked_slider, car
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.75:
        car.throttle = -(speed_slider.value)
        car.steering = 0
    else:
        car.throttle = 0
        time.sleep(1)
        car.throttle = 1 
        time.sleep(1)
        car.throttle = 0
        time.sleep(1)
        car.throttle = -1 
        car.steering = 1
        time.sleep(1)
        car.throttle = 0
        car.steering = 0
        
    
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to initialize

In [7]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [ ]:
prob_blocked = 0
threshold = 0.5

turning = False  # Flag to indicate if the car is turning
turn_time = time.time()  # Start a timer

while True:
    image = camera.read()
    image_proc = preprocess(image).half()
    output = model_trt(image_proc)
    y = F.softmax(output, dim=1)
    prob_blocked = float(y.flatten()[0])

    if prob_blocked < threshold and not turning:
        car.steering = 0.0
    else:
        if not turning:
            car.steering = -0.8  # Choose a steering angle
            turning = True  # Set the turning flag
            turn_time = time.time()  # Start a timer

        if time.time() - turn_time >= 2 and prob_blocked < threshold:
            car.steering = 0.0  # Straighten the wheels
            turning = False  # Clear the turning flag

    car.throttle = 0.18  # Choose car.throttle